# Uniview module for LIGO event GW170817

*Aaron Geller, 2018*

### Imports and function definitions

In [1]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [2]:
Template = """mesh
{  
    data center    ./modules/$folderName/center.raw #I get errors if I don't pass something to geometry??
    data GRB    ./modules/$folderName/GRB.raw
    data grid ./modules/$folderName/grid512.obj
    
    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec2f pfit 1.38504943e-05 3.73532968e-01
            vec1f m1 1.4
            vec1f m2 1.4
            
            
            vec1f eventTime -0.1 | public | desc "event time " | widget slider | range -30 30
            vec1f NSrad 10 | public | desc "NS radius " | widget slider | range 0 100
            #vec1f GWrad 500 | public | desc "GW plane size " | widget slider | range 0 5000
            vec1f kilonovaRad 20000 | public | desc "kilonova max radius " | widget slider | range 0 1e5
            vec1f kilonovaMaxT 0.75 | public | desc "time at max kilonova radius " | widget slider | range 0 1
            vec1f GRBrad 3 | public | desc "GRB point radius " | widget slider | range 0 10
            vec1f GRBspeed 1000 | public | desc "GRB speed " | widget slider | range 0 1000
            vec1f GRBMaxT 1 | public | desc "GRB fade time" | widget slider | range 0 10
            vec1f coronaFac 3 | public | desc "corona radius multiplier" | widget slider | range 0 10

            #for GW membrane
            vec4f membraneColor .3 .3 .6 0.2 | public | desc "GW mesh color" 
            #This chages both the display size and the domain in which the Ricci Scalar is evaluated (e.g. if the value is 100, the curvature is evaluated in the domain where -100 < x < 100, and similarly for y)
            vec1f gridScale 5000  | public| desc "GW grid domain" | widget slider | range 0 1e5
            # This factor controls the contrast between the peaks and valleys of the membrane. Larger values increase contrast.
            vec1f shadingFactor 5000 | public | desc "contrast for GW membrane" | widget slider | range 0 1e5
            #This is an overall amplitude factor
            vec1f amplitudeFactor 1 | public | desc "GW amplitude factor" | widget slider | range 0 1e6
            #The Ricci Scalar Curvature is divergant at the origin. In order to draw attention to the propigation of gravitational waves from the binary system we have added an artifical function of the form exp[-r0^2/r^2] which dramatically attenuates results close to the origin. This parameter is the value of r0 in our attenuation function.
            vec1f centerAttenuationDistance 35 | public | desc "GW amplitude factor" | widget slider | range 0 1000
         }
    }
    ############# gravitation waves
    pass 
    {
        useDataObject grid

        shader
        {
            type defaultMeshShader
            {
                vertexShader ./modules/$folderName/binaryGW.vs
                fragmentShader ./modules/$folderName/binaryGW.fs
                
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.pfit  pfit
                stateManagerVar __objectName__.membraneColor fillColor
                stateManagerVar __objectName__.gridScale gridScale
                stateManagerVar __objectName__.shadingFactor shadingFactor
                stateManagerVar __objectName__.amplitudeFactor A
                stateManagerVar __objectName__.centerAttenuationDistance killFunctionDecay

                glState
                {
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_CULL_FACE_ENABLE false
                }
            }
        }
    }
       

    ############# NS 1 "corona"    
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/corona.gs
                vertexShader ./modules/$folderName/NS.vs
                fragmentShader   ./modules/$folderName/corona.fs
                parameter1f starNum 1. 
                
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.pfit  pfit
                stateManagerVar __objectName__.NSrad  NSrad
                stateManagerVar __objectName__.m1  m1
                stateManagerVar __objectName__.m2  m2
                stateManagerVar __objectName__.coronaFac  coronaFac
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    ############# NS 2 "corona"    
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/corona.gs
                vertexShader ./modules/$folderName/NS.vs
                fragmentShader   ./modules/$folderName/corona.fs
                parameter1f starNum 2. 
                
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.pfit  pfit
                stateManagerVar __objectName__.NSrad  NSrad
                stateManagerVar __objectName__.m1  m1
                stateManagerVar __objectName__.m2  m2
                stateManagerVar __objectName__.coronaFac  coronaFac
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    
    ############# NS 1
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/NS.gs
                vertexShader ./modules/$folderName/NS.vs
                fragmentShader   ./modules/$folderName/NS.fs
                parameter1f starNum 1. 
                
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.pfit  pfit
                stateManagerVar __objectName__.NSrad  NSrad
                stateManagerVar __objectName__.m1  m1
                stateManagerVar __objectName__.m2  m2
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    ############# NS 2
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/NS.gs
                vertexShader ./modules/$folderName/NS.vs
                fragmentShader   ./modules/$folderName/NS.fs
                parameter1f starNum 2. 

                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.pfit  pfit
                stateManagerVar __objectName__.NSrad  NSrad
                stateManagerVar __objectName__.m1  m1
                stateManagerVar __objectName__.m2  m2
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }


    ############# GRB
    pass
    {
        useDataObject GRB
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/GRB.gs
                vertexShader ./modules/$folderName/kilonova.vs
                fragmentShader   ./modules/$folderName/GRB.fs
                
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.GRBrad  GRBrad
                stateManagerVar __objectName__.GRBspeed  GRBspeed
                stateManagerVar __objectName__.GRBMaxT  GRBMaxT
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE
                }
            }

        }    
    }
    ############# kilonova
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/kilonova.gs
                vertexShader ./modules/$folderName/kilonova.vs
                fragmentShader   ./modules/$folderName/kilonova.fs
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }                
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.kilonovaRad  kilonovaRad
                stateManagerVar __objectName__.kilonovaMaxT  kilonovaMaxT
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }

}"""

### Kilonova class

In [3]:
class Kilonova():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("kilonova.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("kilonova.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("kilonova.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("NS.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("NS.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("NS.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("corona.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("corona.fs",dataFolder)      
        uvmod.Utility.ensurerelativepathexsists("binaryGW.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("binaryGW.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("GRB.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("GRB.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [4]:
model = Kilonova(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [9]:
model.object.setcameraradius(1)
model.object.setcoord("Earth")
model.object.setname("Kilonova")
model.object.setguiname("/KavliLecture/Larson/Kilonova")
model.object.settargetradius(20)
model.object.showatstartup(False)
model.cr = 10000
modinfo.setname("Kilonova")
modinfo.setauthor("Aaron Geller<sup>1</sup> Jeffrey SubbaRao, and Shane Larson<sup>2</sup><br />(1)Adler Planetarium,br />(2)Northwestern University")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for LIGO event GW170817")
#modinfo.setthumbnail("data/R0010133.JPG")
modinfo.setversion("1.0")

generator.generate("Kilonova",[],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

## Helper Functions for modifing code
*Reload Module and Shaders in Uniview*

In [6]:
uvmod.Utility.senduvcommand(model.object.name+".reload; system.reloadallshaders")

Unable to connect to Uniview


*Copy modified Shader files and reload*

In [10]:
from config import Settings


In [11]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

### Create colormap texture

In [126]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig("data/cmap.png", bbox_inches='tight',pad_inches=0)

plot_cmap('hot_r')

plt.show()

In [212]:
pfit = [1.38504943e-05, 3.73532968e-01]
t = -0.1
per = (-1.*pfit[0]*t)**pfit[1]; #seconds
print(per, 6./(per*86400))

0.006481031544404803 0.010715029539456128
